In [1]:
#function to write in csv
import csv
def write_list_in_file(final, name):
    with open(name, "w", newline="",encoding="utf8") as fp:
        a = csv.writer(fp, delimiter=',')
        a.writerows(final)

In [2]:
#function to find dominant color from a image
from PIL import Image
import scipy
import scipy.cluster
import scipy.misc
from pprint import pprint

def find_dominant_color(image):
    #image = Image.open(filename)
    #img = cv2.imread('cropped.jpg')
    #image = cv2.blur(img,(5,5))
    NUM_CLUSTERS = 5

    # Convert image into array of values for each point.
    #ar = scipy.misc.fromimage(image)
    ar = cv2.blur(image,(5,5))
    shape = ar.shape

    # Reshape array of values to merge color bands.
    if len(shape) > 2:
        ar = ar.reshape(scipy.product(shape[:2]), shape[2])

    # Get NUM_CLUSTERS worth of centroids.
    codes, _ = scipy.cluster.vq.kmeans(ar.astype(np.float), NUM_CLUSTERS)

    # Pare centroids, removing blacks and whites and shades of really dark and really light.
    original_codes = codes
    for low, hi in [(60, 200), (35, 230), (10, 250)]:
        codes = scipy.array([code for code in codes 
                             if not ((code[0] < low and code[1] < low and code[2] < low) or
                                     (code[0] > hi and code[1] > hi and code[2] > hi))])
        if not len(codes): codes = original_codes
        else: break

    # Assign codes (vector quantization). Each vector is compared to the centroids
    # and assigned the nearest one.
    vecs, _ = scipy.cluster.vq.vq(ar, codes)

    # Count occurences of each clustered vector.
    counts, bins = scipy.histogram(vecs, len(codes))



    # Find the most frequent color, based on the counts.
    index_max = scipy.argmax(counts)
    peak = codes[index_max]
    #print(peak)
    return peak
    # color = ''.join(chr(c) for c in peak).encode('hex')

In [3]:
#function to find video duration
import subprocess
from decimal import *
def get_duration(filename):   
    duration = subprocess.check_output(['ffprobe', '-i', filename, '-show_entries', 'format=duration', '-v', 'quiet', '-of', 'csv=%s' % ("p=0")])
    return Decimal(duration.decode('utf-8'))

In [4]:
def find_brightness(img):
    img_hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
    histr = cv2.calcHist([img_hsv],[2],None,[256],[0,256])
    h,s,v = cv2.split(img_hsv)
    return v.mean()

In [5]:

from imutils import face_utils
import numpy as np
import imutils
import dlib
import cv2
import imageio

def head_pose(image_points,im):
    
    image_points = np.array(image_points,dtype="double") 
    size = im.shape
    #print(image_points)



    model_points = np.array([
                                (0.0, 0.0, 0.0),             # Nose tip
                                (0.0, -330.0, -65.0),        # Chin
                                (-225.0, 170.0, -135.0),     # Left eye left corner
                                (225.0, 170.0, -135.0),      # Right eye right corne
                                (-150.0, -150.0, -125.0),    # Left Mouth corner
                                (150.0, -150.0, -125.0)      # Right mouth corner

                            ])

    # Camera internals

    focal_length = size[1]
    center = (size[1]/2, size[0]/2)
    camera_matrix = np.array(
                             [[focal_length, 0, center[0]],
                             [0, focal_length, center[1]],
                             [0, 0, 1]], dtype = "double"
                             )


    dist_coeffs = np.zeros((4,1)) # Assuming no lens distortion
    (success, rotation_vector, translation_vector) = cv2.solvePnP(model_points, image_points, camera_matrix, dist_coeffs, flags=cv2.SOLVEPNP_ITERATIVE)


    # Project a 3D point (0, 0, 1000.0) onto the image plane.
    # We use this to draw a line sticking out of the nose


    (nose_end_point2D, jacobian) = cv2.projectPoints(np.array([(0.0, 0.0, 1000.0)]), rotation_vector, translation_vector, camera_matrix, dist_coeffs)

#     for p in image_points:
#         print(p)
#         cv2.circle(im, (int(p[0]), int(p[1])), 3, (0,0,255), -1)


    p1 = ( int(image_points[0][0]), int(image_points[0][1]))
    p2 = ( int(nose_end_point2D[0][0][0]), int(nose_end_point2D[0][0][1]))

    slope = (p2[1]-p1[1])/(p2[0]-p1[0])
    
    points = [p1,p2]
    
    return points



In [33]:
im = cv2.imread("./images/images-10000000_1463349307034642_1694941330775474176_n.mp4/Img240.jpg")
print(head_pos([[212, 100], [210, 133], [187, 90], [216, 88], [199, 118], [216, 115]],im))

[ 212.  100.]
[ 210.  133.]
[ 187.   90.]
[ 216.   88.]
[ 199.  118.]
[ 216.  115.]
-0.11764705882352941


In [9]:
import pylab
import imageio
import os
from imutils import face_utils
import numpy as np
import imutils
import dlib
import cv2


path = './video-archieve/natok/Aaj Robibar-1999' #path to videos
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor("./HOG_face_detector/shape_predictor_68_face_landmarks.dat")
for filename in os.listdir(path):
    
    # First Row of Csv
    myTable = []
    myTable.append([])
    myTable[0].append("Frame Number")
    myTable[0].append("Image Name")
    myTable[0].append("Ambient Brightness")
    myTable[0].append("Lip color (RGB)")
    myTable[0].append("Face color (RGB)")
    myTable[0].append("Head Pose (Slope from nose tip)")

    #path to save images cropped from videos
    newpath = './images/natok/Aaj Robibar-1999/'+filename 
    csvpath = './csvs/final_csvs/Aaj Robibar-1999'
    if not os.path.exists(newpath):
        os.makedirs(newpath)
    
    if not os.path.exists(csvpath):
        os.makedirs(csvpath)
    
    #Read Video
    try:
        vid = imageio.get_reader(path+'/'+filename,  'ffmpeg') 
    except:
        continue
    
    #get duration of the video
    duration = get_duration(path+'/'+filename)  
    
    # increment value of frames so that we get one frame per sec
    incrmnt = int(vid.get_length()/duration) 
    
    #initialize
    img_num = 1
    num = 0
    image_points = []
    for i in range(0,6):
        image_points.append([])
    
    #iterate frame
    while(num < vid.get_length()):
        
        try:
            img = vid.get_data(num)
        except:
            num = num+incrmnt 
            continue;
        
        #find brighness of the image
        brightness = find_brightness(img)
        
        #Face Detection
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        rects = detector(gray, 1)
        
        #for each shape iterate
        for (i, rect) in enumerate(rects):
            
            shape = predictor(gray, rect)
            shape = face_utils.shape_to_np(shape) 
            
            #get face bounding rectangle
            (x, y, w, h) = face_utils.rect_to_bb(rect)
            try:
                
                #crop image
                cropped = img[y-30:y + h+30, x-30:x + w+30].copy()
                
                #save cropped image
                imageio.imwrite( newpath+"/Img"+str(img_num)+".jpg", cropped);
                
                #img_num = img_num+1
                
                #loop over the face parts individually
                for (name, (i, j)) in face_utils.FACIAL_LANDMARKS_IDXS.items():
                    
                    try:
                        (x, y, w, h) = cv2.boundingRect(np.array([shape[i:j]]))
                        cropped = img[y:y + h, x:x + w].copy()

                        if(name == "jaw"):
                            
                            #chin point
                            image_points[1] =[shape[i:j][8][0],shape[i:j][8][1]]
                            
                            #find face color
                            face_color = find_dominant_color(cropped)
                            #myTable[img_num].append(face_color)

                        elif(name == "mouth"):
                            
                            #left and right marker of lips
                            image_points[4]=[shape[i:j][0][0],shape[i:j][0][1]]
                            image_points[5]=[shape[i:j][6][0],shape[i:j][6][1]]
                            
                            #find lip color
                            lip_color = find_dominant_color(cropped)
                            #myTable[img_num].append(lip_color)
                            
                        elif(name == "right_eye"):
                            
                            #this is actually the left eye. left corner of left eye
                            image_points[2]=[shape[i:j][0][0],shape[i:j][0][1]]
                            
                        elif(name == "left_eye"):
                            
                            #and this is the right eye.right corner of right eye
                            image_points[3]=[shape[i:j][3][0],shape[i:j][3][1]]
                            
                        elif(name == "nose"):
                            
                            #nose tip
                            image_points[0]=[shape[i:j][3][0],shape[i:j][3][1]]

                    except ValueError:
                        print("error khaise")
                        continue;
                #print(image_points)        
                slope = head_pose(image_points,img)
                #print(slope)
                myTable.append([])
                myTable[img_num].append(num)
                myTable[img_num].append(img_num)
                myTable[img_num].append(brightness)
                myTable[img_num].append(lip_color)
                myTable[img_num].append(face_color)
                myTable[img_num].append(slope)
                img_num = img_num+1
                #print(myTable[img_num])
            except Exception:
                #img_num = img_num+1
                print(Exception)
                continue;
                
        ##one frame per one sec.
        num = num+incrmnt  
        
    write_list_in_file(myTable, csvpath +'/result_'+filename+'.csv')

<class 'Exception'>
<class 'Exception'>
<class 'Exception'>
<class 'Exception'>
<class 'Exception'>
<class 'Exception'>
<class 'Exception'>
<class 'Exception'>
<class 'Exception'>
<class 'Exception'>
<class 'Exception'>
<class 'Exception'>
<class 'Exception'>
<class 'Exception'>
<class 'Exception'>
<class 'Exception'>
<class 'Exception'>
<class 'Exception'>
<class 'Exception'>
<class 'Exception'>
<class 'Exception'>
<class 'Exception'>
<class 'Exception'>
<class 'Exception'>
<class 'Exception'>
<class 'Exception'>
<class 'Exception'>
<class 'Exception'>
<class 'Exception'>
<class 'Exception'>
<class 'Exception'>
<class 'Exception'>
<class 'Exception'>
<class 'Exception'>
<class 'Exception'>
<class 'Exception'>
<class 'Exception'>
<class 'Exception'>
<class 'Exception'>
<class 'Exception'>
<class 'Exception'>
<class 'Exception'>
<class 'Exception'>
<class 'Exception'>
<class 'Exception'>
<class 'Exception'>
<class 'Exception'>
<class 'Exception'>
<class 'Exception'>
<class 'Exception'>


In [8]:
write_list_in_file(myTable,'result_tahm1.csv')

In [15]:
import pylab
import imageio
import os

newpath = './images/images-'+filename #path to save images cropped from videos
lipspath = newpath+'/lips' #path to save images cropped from videos
#image = cv2.imread("img9.jpg")
print(find_dominant_color(lipspath+"/Img126.jpg"))
#print(find_dominant_color(lipspath+"/Img1.jpg"))
#print(find_dominant_color("Img4.jpg"))

[ 114.97916667   73.01388889   58.65972222]


In [6]:
def rect_to_bb(rect):

    x = rect.left()
    y = rect.top()
    w = rect.right() - x
    h = rect.bottom() - y
 
    # return a tuple of (x, y, w, h)
    return (x, y, w, h)

In [12]:
from imutils import face_utils
import numpy as np
import imutils
import dlib
import cv2
import imageio
# initialize dlib's face detector (HOG-based) and then create
# the facial landmark predictor
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor("HOG_face_detector/shape_predictor_68_face_landmarks.dat")
 
# load the input image, resize it, and convert it to grayscale
image = cv2.imread("example_images/sachin.jpg")
#cv2.imshow("image",image)
#image = imutils.resize(image, width=500)
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
 
# detect faces in the grayscale image
rects = detector(gray, 1)

# loop over the face detections
for (i, rect) in enumerate(rects):
    # determine the facial landmarks for the face region, then
    # convert the landmark (x, y)-coordinates to a NumPy array
    #shape = predictor(gray, rect)
    #shape = face_utils.shape_to_np(shape)
    (x, y, w, h) = rect_to_bb(rect)
    cropped = image[y:w+h,x:x+w].copy()
    imageio.imwrite("1.jpg",cropped)
 
    # loop over the face parts individually
#     for (name, (i, j)) in face_utils.FACIAL_LANDMARKS_IDXS.items():
#         # clone the original image so we can draw on it, then
#         # display the name of the face part on the image
#         clone = image.copy()
#         cv2.putText(clone, name, (10, 30), cv2.FONT_HERSHEY_SIMPLEX,
#             0.7, (0, 0, 255), 2)
 
#         # loop over the subset of facial landmarks, drawing the
#         # specific face part
#         for (x, y) in shape[i:j]:
#             cv2.circle(clone, (x, y), 1, (0, 0, 255), -1)
            
#         # extract the ROI of the face region as a separate image
#         (x, y, w, h) = cv2.boundingRect(np.array([shape[i:j]]))
#         roi = image[y:y + h, x:x + w]
#         roi = imutils.resize(roi, width=250, inter=cv2.INTER_CUBIC)
 
#         # show the particular face part
#         #plt.imshow(clone)
#         cv2.imshow("ROI", roi)
#         imageio.imwrite( "1.jpg", roi); #save the cropped image
#         print(find_dominant_color("1.jpg"))
#         cv2.imshow("Image", clone)
#         cv2.waitKey(0)
 
#     # visualize all facial landmarks with a transparent overlay
#     output = face_utils.visualize_facial_landmarks(image, shape)
#     cv2.imshow("Image", output)
#     cv2.waitKey(0)
   

In [5]:
import pylab
import imageio
import os
from imutils import face_utils
import numpy as np
import imutils
import dlib
import cv2


path = './video-archieve/test' #path to videos
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor("./HOG_face_detector/shape_predictor_68_face_landmarks.dat")
for filename in os.listdir(path):
    newpath = './images/images-'+filename #path to save images cropped from videos
    lipspath = newpath+'/lips' #path to save images cropped from videos
    if not os.path.exists(newpath):
        os.makedirs(newpath)
        
    if not os.path.exists(lipspath):
        os.makedirs(lipspath)
    
    vid = imageio.get_reader(path+'/'+filename,  'ffmpeg') #Read Video
    img_num=0
    num = 0
    duration = get_duration(path+'/'+filename)  #get duration of the video
    incrmnt = int(vid.get_length()/duration)  # increment value of frames so that we get one frame per sec
    frame_index = []
    brightness_arr= []
    #ncrmnt = 1000
    while(num < vid.get_length()):
        #Face Detection
        img = vid.get_data(num)
        #brightness = find_brightness(img)
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        
        num = num+incrmnt ##one frame per one sec. 
#     # First Row of Csv
#     myTable = []
#     myTable.append([])
#     myTable[0].append("Frame Number")
#     myTable[0].append("Image Name")
#     myTable[0].append("Gender")
#     myTable[0].append("RGB")
#     myTable[0].append("Brightness")
#     #iterate images
#     i=1
#     for index in frame_index:
#         myTable.append([])
#         myTable[i].append(str(index))
#         myTable[i].append(str(i-1))   
#         example_image = './'+newpath+'/Img'+str(i-1)+'.jpg'
#         input_image = caffe.io.load_image(example_image)
#         #predict gender
#         prediction = gender_net.predict([input_image]) 
#         myTable[i].append(str(gender_list[prediction[0].argmax()]))
#         #find dominant color from the cropped image i.e face color
#         res = find_dominant_color(example_image)
#         myTable[i].append(str(res[0])+','+str(res[1])+','+str(res[2]))
#         myTable[i].append(str(brightness_arr[i-1]))
#         i = i+1
#     #Write the results in csv 
#     write_list_in_file(myTable,filename+'result.csv')
